# EDA


- The os module has a perfect method to list files in a directory.
- Pandas json normalize could work here but is not necessary to convert the JSON data to a dataframe.
- You may need a nested for-loop to access each sale!
- We've put a lot of time into creating the structure of this repository, and it's a good example for future projects.  In the file functions_variables.py, there is an example function that you can import and use.  If you have any variables, functions or classes that you want to make, they can be put in the functions_variables.py file and imported into a notebook.  Note that only .py files can be imported into a notebook. If you want to import everything from a .py file, you can use the following:
```python
from functions_variables import *
```
If you just import functions_variables, then each object from the file will need to be prepended with "functions_variables"\
Using this .py file will keep your notebooks very organized and make it easier to reuse code between notebooks.

In [ ]:
# (this is not an exhaustive list of libraries)
import pandas as pd
import numpy as np
import os
import json
from pprint import pprint
from functions_variables import encode_tags
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from category_encoders import TargetEncoder

## Data Importing

In [ ]:
# load one file first to see what type of data you're dealing with and what attributes it has

# Define the data directory and file path
current_dir = os.getcwd()
main_dir = os.path.abspath(os.path.join(current_dir, '..'))
data_dir = os.path.join(main_dir, 'data')
sample_file = os.path.join(data_dir, "AK_Juneau_0.json")

# Load JSON data
with open(sample_file, "r") as f:
    sample_data = json.load(f)

# Convert JSON to a DataFrame
json_structure = pd.DataFrame(sample_data)

# Creating a file for csv data:
csv_dir = os.path.join(main_dir, 'csv_data')
os.makedirs(csv_dir, exist_ok=True)

# Saving json_structure df to csv: 
json_structure.to_csv(os.path.join(csv_dir, 'json_structure.csv'), index=False)
print("CSV file created successfully!")

In [ ]:
# Print a preview of the data
print(json.dumps(sample_data, indent=4))

In [ ]:
# loop over all files and put them into a dataframe

# Initialize a list to store extracted sale records
data_list = []

# Loop through each JSON file
for file in os.listdir(data_dir):
    if not file.endswith('.json'):
        continue
        
    file_path = os.path.join(data_dir, file)  # Construct full file path
    
    with open(file_path, "r") as f:
        raw_data = json.load(f)  # Load JSON data
        
        # Extract property listings (assuming structure is in "data" -> "results")
        listings = raw_data.get("data", {}).get("results", [])

        if not isinstance(listings, list):  # Check if "results" is not a list
            print(f"Skipping malformed file: {file}")
            continue  # Skip files without proper listings

        # Process each listing
        for listing in listings:
            sale_record = {
                "property_id": listing.get("property_id", "Unknown"),
                "permalink": listing.get("permalink", "Unknown"),
                "status": listing.get("status", "Unknown"),
                "year_built": listing.get("description", {}).get("year_built", None),
                "garage": listing.get("description", {}).get("garage", None),
                "stories": listing.get("description", {}).get("stories", None),
                "beds": listing.get("description", {}).get("beds", None),
                "baths_1qtr": listing.get("description", {}).get("baths_1qtr", None),
                "baths_3qtr": listing.get("description", {}).get("baths_3qtr", None),
                "baths_half": listing.get("description", {}).get("baths_half", None),
                "baths_full": listing.get("description", {}).get("baths_full", None),
                "baths": listing.get("description", {}).get("baths", "Unknown"),
                "type": listing.get("description", {}).get("type", "Unknown"),
                "sub_type": listing.get("description", {}).get("sub_type", "Unknown"),
                "lot_sqft": listing.get("description", {}).get("lot_sqft", None),
                "sqft": listing.get("description", {}).get("sqft", None),
                "sold_price": listing.get("description", {}).get("sold_price", None),
                "sold_date": pd.to_datetime(listing.get("description", {}).get("sold_date", "Unknown")),
                "list_price": listing.get("list_price", None),
                "last_update_date": pd.to_datetime(listing.get("last_update_date"), errors='coerce'),
                "city": listing.get("location", {}).get("address", {}).get("city", "Unknown"),
                "state": listing.get("location", {}).get("address", {}).get("state", "Unknown"),
                "postal_code": listing.get("location", {}).get("address", {}).get("postal_code", "Unknown"),
                "street_view_url": listing.get("location", {}).get("street_view_url", "Unknown"),
                "tags": listing.get("tags", [])  
            }
            data_list.append(sale_record)


# Convert extracted data into a DataFrame
listings_df = pd.DataFrame(data_list)

# Save to CSV for easier analysis
listings_df.to_csv(os.path.join(csv_dir, "raw_real_estate_listings.csv"), index=False)
print("CSV file created successfully!")

## Data Cleaning and Wrangling

At this point, ensure that you have all sales in a dataframe.
- Take a quick look at your data (i.e. `.info()`, `.describe()`) - what do you see?
- Is each cell one value, or do some cells have lists?
- What are the data types of each column?
- Some sales may not actually include the sale price (target).  These rows should be dropped.
- There are a lot of NA/None values.  Should these be dropped or replaced with something?
    - You can drop rows or use various methods to fills NA's - use your best judgement for each column 
    - i.e. for some columns (like Garage), NA probably just means no Garage, so 0
- Drop columns that aren't needed
    - Don't keep the list price because it will be too close to the sale price. Assume we want to predict the price of houses not yet listed

In [ ]:
listings_df.info()

At first glance, `baths_1qtr` has all null values and should be dropped.

In [ ]:
listings_df.describe()

In [ ]:
# Loading and inspecting the data
listings_df

In [ ]:
# Dropping entrites with duplicate 'property_id'

listings_df = listings_df.drop_duplicates(subset='property_id')

In [ ]:
# Modifying datatypes

# float to int
listings_df.loc[:, 'year_built'] = listings_df['year_built'].astype('Int64') 
listings_df.loc[:, 'garage'] = listings_df['garage'].astype('Int64')
listings_df.loc[:, 'stories'] = listings_df['stories'].astype('Int64')
listings_df.loc[:, 'beds'] = listings_df['beds'].astype('Int64')
listings_df.loc[:, 'baths'] = listings_df['baths'].astype('Int64')

# Remove time from last_update_date and convert to datetime64
listings_df.loc[:, 'last_update_date'] = pd.to_datetime(listings_df['last_update_date'].astype('str').str[:10], errors='coerce')

In [ ]:
# Addressing Null values 

pd.DataFrame(listings_df.isnull().sum() * 100 / len(listings_df)).reset_index().sort_values(0, ascending=False).rename(columns={'index': 'column_name',
                                                                                                              0: 'percent_missing'})

In [ ]:
# Dropping the 4 columns wehere over 50% of the data is null (baths_1qtr)
listings_df = listings_df.drop(columns=['baths_1qtr','baths_3qtr', 'sub_type', 'baths_half'])

# Dropping the redundant 'baths_full' column in favor of the 'baths' column
listings_df = listings_df.drop(columns='baths_full')

# Dropping the 'list_price' column in favor of 'sold_price' to predict the prices of houses not listed
listings_df = listings_df.drop(columns='list_price')

# Dropping the rows that have a null sale price (target)
listings_df = listings_df[~listings_df['sold_price'].isnull()]

In [ ]:
# To suppress the warning that results from filling last_update_date 
pd.set_option('future.no_silent_downcasting', True)

In [ ]:
# Null value replacement

# if none then 0: 
listings_df['garage'] = listings_df['garage'].fillna(value=0) 
listings_df['lot_sqft'] = listings_df['lot_sqft'].fillna(value=0)
listings_df['sqft'] = listings_df['sqft'].fillna(value=0)
listings_df['beds'] = listings_df['beds'].fillna(value=0)
listings_df['baths'] = listings_df['baths'].fillna(value=0)
listings_df['tags'] = listings_df['tags'].fillna(value='[]')

listings_df['last_update_date'] = listings_df['last_update_date'].fillna(value=listings_df['sold_date']) # Assuming the listing was last updated on the day it sold 
print('null type count:', listings_df[listings_df['type'].isnull() ==True].shape[0])
listings_df['type'] = listings_df['type'].fillna(value='single_family') # Only a few entries, validated type manually through zillow search 

print('null city count:', listings_df[listings_df['city'].isnull() ==True].shape[0])
listings_df['city'] = listings_df['city'].fillna(value='Columbus') # Only one entry, validated city manually through search 

listings_df['stories'] = listings_df['stories'].fillna(value=0) # Manually validated some addresses with stories null, appears to be warehouses, fields, etc

In [ ]:
# Null year_built:

null_years_built = listings_df[listings_df['year_built'].isnull() == True]
null_years_built

In [ ]:
null_years_built.groupby('type').count()

In [ ]:
listings_df.groupby(by=['type', 'city'])['year_built'].min()['land']

In [ ]:
listings_df.groupby(by=['type', 'city'])['year_built'].min()['condo']

The majority of records that have null `year_built` are of `land` or `single_family` type. For those of `single_family` type, values will be grouped by type and city, and the most frequently occured year will be the replacement. Only one record of 'year_built' exists for the listings of land type, the year 1956 in Phoenix, Arizona. This value is used to impute records with null 'year_built' that is of land type. It is important to note that this imputation will reduce the variability of the data. A similar issue exists for those of the condo type, with the only record of the year 1985 in Nashville. The same imputation method will be used. The two entries that are of type 'other' are the only entries in the dataframe of that type, and upon manual inspection the listings are for the lot or incomplete so they will be dropped. After the conditions are applied, there are only 3 records left, whose year_built information cannot be found online. To maintain more data integretity, these records will be dropped instead of trying to find a statristic to genneralize. 

In [ ]:
listings_df[listings_df['year_built'].isnull() == True]

In [ ]:
# Replace null values for 'year_built'

# Define conditions 
condition_sf = (listings_df['type'] == 'single_family') & (listings_df['year_built'].isna())
condition_land = (listings_df['type'] == 'land') & (listings_df['year_built'].isna())
condition_condo = (listings_df['type'] == 'condo') & (listings_df['year_built'].isna())
condition_mobile = (listings_df['type'] == 'mobile') & (listings_df['year_built'].isna())
# Manual search conditions
condition_ms1 = (listings_df['property_id'] == '5328654220') & (listings_df['year_built'].isna())   
condition_ms2 = (listings_df['property_id'] == '9414271198') & (listings_df['year_built'].isna())

condition_ms3 = (listings_df['property_id'] == '9988039199') & (listings_df['year_built'].isna())
condition_ms4 = (listings_df['property_id'] == '3377433199') & (listings_df['year_built'].isna())

# Group by type and city, then get mode of rows of single family type
sf_modes = (listings_df[listings_df['type'] == 'single_family'].groupby(['type', 'city'])['year_built'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else pd.NA))

# Define a function to get the mode from the groupby
def get_mode(row):
    try:
        return sf_modes.loc[(row['type'], row['city'])]
    except KeyError:
        return row['year_built']  # fallback if no mode available

# Apply the function to the single_family condition
listings_df.loc[condition_sf, 'year_built'] = listings_df[condition_sf].apply(get_mode, axis=1)
print('null type count after single_family modes:', listings_df[listings_df['year_built'].isnull() ==True].shape[0])

# Manually validating and imputation everything else
listings_df.loc[condition_land, 'year_built'] = 1956
listings_df.loc[condition_condo, 'year_built'] = 1985
listings_df = listings_df[listings_df['type'] != 'other']
listings_df.loc[condition_mobile, 'year_built'] = 1988

# Values obtained through manual search
listings_df.loc[condition_ms1, 'year_built'] = 1876
listings_df.loc[condition_ms2, 'year_built'] = 2023
listings_df.loc[condition_ms3, 'year_built'] = 1983
listings_df.loc[condition_ms4, 'year_built'] = 1981
print('null type count after manual conditions:', listings_df[listings_df['year_built'].isnull() ==True].shape[0])

# Dropping remaining rows
listings_df = listings_df[listings_df['year_built'].isnull() == False]

In [ ]:
# Drop remaining redundant columns 
listings_df = listings_df.drop(columns=['permalink', 'postal_code', 'street_view_url'])

# Checking that all listings are confirmed to be sold before dropping the status column
print('Listing statuses:')
print(listings_df['status'].unique())
listings_df = listings_df.drop(columns='status')

In [ ]:
# Validate cleaned dataframe and save to csv
display(listings_df.head())
display(listings_df.info())

listings_df.to_csv(os.path.join(csv_dir, "cleaned_real_estate_listings.csv"), index=False)
print("CSV file created successfully!")

In [ ]:
print(f"Number of columns: {listings_df.shape[1]}")

In [ ]:
# Feature engineering

listings_df['total_sqft'] = sum(listings_df['lot_sqft'], listings_df['sqft'])
listings_df['building_ratio'] = listings_df['sqft'] / listings_df['lot_sqft'] # higher ratio = more building area(urban), 0 = all lot no building
listings_df['building_ratio'] = listings_df['building_ratio'].replace([np.inf, -np.inf], np.nan) # fixing cases where ratio is inf (0 sqft and positive lot sqft) 
listings_df['building_ratio'] = listings_df['building_ratio'].fillna(value=listings_df['building_ratio'].mean()) # filling nulls with global building ratio

### Dealing with Tags

Consider the fact that with tags, there are a lot of categorical variables.
- How many columns would we have if we OHE tags, city and state?
- Perhaps we can get rid of tags that have a low frequency.

In [ ]:
# OHE categorical variables/ tags here
# tags will have to be done manually

In [ ]:
# Estimate the Number of Columns Created by OHE
print(f"Unique tags: {listings_df['tags'].explode().nunique()}")
print(f"Unique cities: {listings_df['city'].nunique()}")
print(f"Unique states: {listings_df['state'].nunique()}")

In [ ]:
# Encode tags
listings_df = encode_tags(listings_df, min_occurrences=100)

# Dropping original tag column 
listings_df = listings_df.drop(columns='tags')

# Check updated column count
print(f"Updated column count: {len(listings_df.columns)}")

### Dealing with Cities

- Sales will vary drastically between cities and states.  Is there a way to keep information about which city it is without OHE?
- Could we label encode or ordinal encode?  Yes, but this may have undesirable effects, giving nominal data ordinal values.
- What we can do is use our training data to encode the mean sale price by city as a feature (a.k.a. Target Encoding)
    - We can do this as long as we ONLY use the training data - we're using the available data to give us a 'starting guess' of the price for each city, without needing to encode city explicitly
- If you replace cities or states with numerical values (like the mean price), make sure that the data is split so that we don't leak data into the training selection. This is a great time to train test split. Compute on the training data, and join these values to the test data
- Note that you *may* have cities in the test set that are not in the training set. You don't want these to be NA, so maybe you can fill them with the overall mean

In [ ]:
# perform train test split here
from category_encoders import TargetEncoder

# Using k-fold CV with shuffle for a dataset this size 
X = listings_df.drop(columns='sold_price')
y = listings_df['sold_price']

kf = KFold(n_splits=5, shuffle=True, random_state=13)

# Isolating train/test dataframes from k-fold split
for train_idx, test_idx in kf.split(X):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    # Initiating target encoder and only transforming the training set     
    encoder = TargetEncoder(cols=['city', 'state'], smoothing=3)
    encoded_data = encoder.fit_transform(X_train[['city', 'state']], y_train)

    # Inserting encoded data into main dataframe
    listings_df['encoded_city'] = round((encoded_data['city']), 2) 
    listings_df['encoded_state'] = round((encoded_data['state']), 2) 

In [ ]:
# Viewing encoded city and state columns side by side to validate

city_encodings = listings_df[['city', 'encoded_city']].dropna(subset=['encoded_city']).drop_duplicates('city').sort_values(by='encoded_city', ascending=False)
state_encodings = listings_df[['state', 'encoded_state']].dropna(subset=['encoded_state']).drop_duplicates('state').sort_values(by='encoded_state', ascending=False)

display(city_encodings.head(10))
display(state_encodings.head(10))

In [ ]:
# Use encoded values to fill in test set 
city_mappings = city_encodings.set_index('city')['encoded_city']
state_mappings = state_encodings.set_index('state')['encoded_state']

listings_df['encoded_city'] = listings_df['encoded_city'].fillna(listings_df['city'].map(city_mappings))
listings_df['encoded_state'] = listings_df['encoded_state'].fillna(listings_df['state'].map(state_mappings))

In [ ]:
# Check for nulls, replace with global city/state means if exist 

overall_mean_price = listings_df["sold_price"].mean()
listings_df["encoded_city"] = listings_df["encoded_city"].fillna(overall_mean_price)
listings_df["encoded_state"] = listings_df["encoded_state"].fillna(overall_mean_price)
listings_df["encoded_city"] = listings_df["encoded_city"].astype('float')
listings_df["encoded_state"] = listings_df["encoded_state"].astype('float')

listings_df[['encoded_city', 'encoded_state']].info()

In [ ]:
# Encoding remaining categorical columns 

# Conditions for encoding type
type1 = listings_df['type'] == 'mobile'
type2 = listings_df['type'] == 'apartment' 
type3 = listings_df['type'] == 'condo'
type4 = listings_df['type'] == 'condos'
type5 = listings_df['type'] == 'condo_townhome_rowhome_coop'
type6 = listings_df['type'] == 'townhomes'
type7 = listings_df['type'] == 'single_family'
type8 = listings_df['type'] == 'duplex_triplex'
type9 = listings_df['type'] == 'multi_family'
type10 = listings_df['type'] == 'land'

listings_df.loc[type1, 'type'] = 1.0
listings_df.loc[type2, 'type'] = 2
listings_df.loc[type3, 'type'] = 3
listings_df.loc[type4, 'type'] = 4
listings_df.loc[type5, 'type'] = 5
listings_df.loc[type6, 'type'] = 6
listings_df.loc[type7, 'type'] = 7
listings_df.loc[type8, 'type'] = 8
listings_df.loc[type9, 'type'] = 9
listings_df.loc[type10, 'type'] = 10
listings_df['type'] = listings_df['type'].astype('Int64')      

In [ ]:
listings_df.select_dtypes(include=object) # drop property id and last update date? 

Many listings of the 'mobile' type have 0 sqft in total after feature engineering

In [ ]:
# Save encoded dataset 
listings_df.to_csv(os.path.join(csv_dir, "encoded_real_estate_listings.csv"), index=False)

print("Encoded dataset saved successfully!")

## Extra Data - STRETCH

> This doesn't need to be part of your Minimum Viable Product (MVP). We recommend you write a functional, basic pipeline first, then circle back and join new data if you have time

> If you do this, try to write your downstream steps in a way it will still work on a dataframe with different features!

- You're not limited to just using the data provided to you. Think/ do some research about other features that might be useful to predict housing prices. 
- Can you import and join this data? Make sure you do any necessary preprocessing and make sure it is joined correctly.
- Example suggestion: could mortgage interest rates in the year of the listing affect the price? 

In [ ]:
# import, join and preprocess new data here

## EDA/ Visualization

Remember all of the EDA that you've been learning about?  Now is a perfect time for it!
- Look at distributions of numerical variables to see the shape of the data and detect outliers.    
    - Consider transforming very skewed variables
- Scatterplots of a numerical variable and the target go a long way to show correlations.
- A heatmap will help detect highly correlated features, and we don't want these.
    - You may have too many features to do this, in which case you can simply compute the most correlated feature-pairs and list them
- Is there any overlap in any of the features? (redundant information, like number of this or that room...)

In [ ]:
cat_cols = listings_df[['type', 'city', 'state']]
num_cols = listings_df[['year_built', 'garage', 'stories', 'beds', 'baths', 'lot_sqft', 'sqft', 'sold_price', 'total_sqft', 'building_ratio']]

In [ ]:
box_cols = listings_df[['garage', 'stories', 'beds', 'baths']]

In [ ]:
for column in num_cols:
    sns.displot(listings_df[column])

In [ ]:
for column in box_cols: 
    sns.boxplot(listings_df[[column]])

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Histogram to check distributions
test_df.hist(figsize=(20, 8), bins=30)
plt.show()

# Boxplot to detect outliers
sns.boxplot(data=listings_df[['year_built', 'beds', 'baths']])
plt.show()


In [ ]:
state_avg_price = listings_df.groupby("state")["sold_price"].mean().reset_index()

plt.figure(figsize=(12, 6))
sns.barplot(x="sold_price", y="state", data=state_avg_price)
plt.xticks(rotation=90)
plt.title("Average Sale Price by State")
plt.show()

In [ ]:
# Scatter plot
plt.figure(figsize=(10, 6))
sns.scatterplot(x=listings_df["year_built"], y=listings_df["sqft"])
plt.xlabel("Year Built")
plt.ylabel("square feet")
plt.title("Square Feet vs. Year built")
plt.show()

In [ ]:
# Scatter plot
plt.figure(figsize=(10, 6))
sns.scatterplot(x=listings_df["sqft"], y=listings_df["sold_price"])
plt.xlabel("Square Feet")
plt.ylabel("Sale Price")
plt.title("Square Feet vs. Sale Price")
plt.show()

In [ ]:
# Scatter plot
plt.figure(figsize=(10, 6))
sns.scatterplot(x=listings_df["total_sqft"], y=listings_df["sold_price"])
plt.xlabel("Square Feet")
plt.ylabel("Sale Price")
plt.title("Square Feet vs. Sale Price")
plt.show()

In [ ]:
sns.regplot(x=listings_df["sqft"], y=listings_df["sold_price"])

In [ ]:
sns.regplot(x=listings_df["encoded_state"], y=listings_df["sold_price"])

In [ ]:
sns.heatmap(listings_df[['year_built', 'garage', 'stories', 'beds', 'baths', 'lot_sqft', 'sqft', 'sold_price', 'encoded_city', 'encoded_state', 'total_sqft', 'building_ratio']].corr(numeric_only=True), annot=True)

## Scaling and Finishing Up

Now is a great time to scale the data and save it once it's preprocessed.
- You can save it in your data folder, but you may want to make a new `processed/` subfolder to keep it organized

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Choose scaling method
scaler = StandardScaler()  # Use MinMaxScaler() if needed

# Select numeric columns for scaling
cols_to_scale = ['lot_sqft', 'sqft', 'total_sqft', 'building_ratio', 'encoded_city', 'encoded_state']
listings_df[cols_to_scale] = scaler.fit_transform(listings_df[cols_to_scale])

In [ ]:
# Define file path for processed data 
processed_dir = os.path.join(main_dir, 'processed_data')
os.makedirs(processed_dir, exist_ok=True)

# Save the processed dataset
processed_listings = listings_df.drop(columns=['city', 'state'])
processed_listings.to_csv(os.path.join(processed_dir, 'processed_real_estate_listings.csv'), index=False)
print("Processed dataset saved successfully!")

# Train/test split on processed dataset and save to csv
X = processed_listings.drop(columns='sold_price')
y = processed_listings['sold_price']

kf = KFold(n_splits=5, shuffle=True, random_state=13)

for train_idx, test_idx in kf.split(X):
    train_df = X.iloc[train_idx].copy()
    test_df = X.iloc[test_idx].copy()

train_df.to_csv(os.path.join(processed_dir, 'train_df.csv'), index=False)
test_df.to_csv(os.path.join(processed_dir, 'test_df.csv'), index=False)
print("Train/test datasets saved successfully!")